In [2]:
import numpy as np
import networkx as nx
import os
import dwave_networkx as dnx
import dimod

In [3]:
def ising_to_qubo(J):
  n = J.shape[0]
  Q = np.zeros((n, n))
  # Jijsisj = Jij(2*xi-1)(2*xj-1)=4Jijxixj-2Jijxi-2ijxj
  nnzq = 0
  offset = 0
  for u in range(n):
      for v in range(u+1, n):
         coupling = J[u,v]
         if abs(coupling) > 0:
          Q[u,v] = 2*coupling
          Q[v,u] = 2*coupling
          nnzq += 1
          Q[u,u] -= coupling
          Q[v,v] -= coupling
          offset+=coupling
  return Q, nnzq, offset
def write_qubo(path: str, Q: np.ndarray, nnzquad: int):
   n = Q.shape[0]
   with open(path, 'w') as qubofile:
      qubofile.write(f'p qubo 0 {Q.shape[0]} {Q.shape[0]} {nnzquad}\n')
      for u in range(n):
          coupling = Q[u,u]
          if abs(coupling) > 0:
              qubofile.write(f'{u} {u} {coupling}\n')
      for u in range(n):
          for v in range(u+1, n):
            coupling = Q[u,v]
            if abs(coupling) > 0:
                qubofile.write(f'{u} {v} {coupling}\n')

In [4]:
def readgraph(path: str):
  with open(path, mode='r') as gfile:
    g = nx.Graph()
    nodes, edges = [int(i) for i in gfile.readline().strip().split()]
    elist = []
    for line in gfile.readlines():
      args = line.strip().split()
      if len(args) < 2:
        continue
      u,v = [int(i) for i in args[0:2]]
      u-=1
      v-=1
      if len(args) < 3:
        elist.append((u,v,1.0))
      else:
        elist.append((u,v,float(args[2])))
    g.add_weighted_edges_from(elist)
    return g



In [159]:
g = readgraph(os.environ['GSET']+'/set/G022')
a = nx.to_numpy_array(g, nodelist=np.arange(g.number_of_nodes()))
J = a
Q, nnz, offset = ising_to_qubo(J)
write_qubo('G022.qubo', Q, nnz)

In [168]:
for q in Q:
  print('{', end='')
  for qi in q[:-1]:
    print(qi, end=', ')
  print('%.1f},' % (q[-1]))

{-5.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0},
{0.0, -2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0},
{0.0, 0.0, -2.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0},
{2.0, 0.0, 0.0, -2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0},
{0.0, 2.0, 2.0, 0.0, -5.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0},
{0.0, 0.0, 0.0, 0.0, 2.0, -3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0},
{0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 2.0},
{2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, -5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 0.0},
{2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0},
{0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0},
{0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, -2.0, 0.0, 0.0, 0.0, 0.0},
{2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, -3.0, 0.0, 0.0, 0.0},
{0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, -1.

In [104]:
spins = np.random.choice([-1,1], 800)
0.5 * (spins.transpose() @ J @ spins)

-46.0

In [105]:
x = (spins + 1) / 2
x.transpose() @ Q @ x + offset

-46.0

In [122]:
Q

array([[-1.88168677,  1.80961813, -0.27845159,  1.90996079, -1.39662361,
         0.06727913,  0.37755264,  0.3047176 , -0.58439034, -1.6030421 ,
        -1.70281505,  1.64168758,  1.03318305,  1.80944109,  0.43424397,
        -1.94067449],
       [ 1.80961813, -0.29952201, -0.53524311, -0.32635377, -1.54477708,
        -1.88672698,  0.93962561, -0.96053254,  1.88866038,  0.27354911,
         0.28285362, -2.        ,  1.45269118,  1.60290483, -0.86729799,
         0.17055064],
       [-0.27845159, -0.53524311, 11.94870473, -0.79992883, -0.67204552,
        -0.69539519, -1.42925913, -0.64619993, -1.30965022, -1.23260744,
        -1.17921149, -0.25695846, -0.67106778,  0.06258753, -1.48803437,
        -0.8172392 ],
       [ 1.90996079, -0.32635377, -0.79992883,  3.95513539, -1.00342603,
        -1.65478078, -1.98585252,  0.93546873, -1.33456827, -1.03426197,
         0.9917478 , -0.0849514 , -1.34958485,  0.83900461, -0.89196336,
         1.83435444],
       [-1.39662361, -1.54477708, -0

In [120]:
bqmtest = dimod.BQM.from_qubo(Q)

In [171]:
sampler = dimod.SimulatedAnnealingSampler()
sampler.sample_qubo(Q)

SampleSet(rec.array([([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0], -17., 1),
           ([0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1], -18., 1),
           ([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1], -18., 1),
           ([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1], -18., 1),
           ([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1], -18., 1),
           ([1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0], -17., 1),
           ([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1], -18., 1),
           ([1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0], -18., 1),
           ([1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0], -18., 1),
           ([0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1], -18., 1)],
          dtype=[('sample', 'i1', (15,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables(range(0, 15)), {}, 'BINARY')

In [169]:
bitstr = '000110011011011'.replace(', ', '')

In [170]:
x = np.array([int(i) for i in bitstr])
s = [i for i,c in enumerate(bitstr) if c == '1']
nx.cut_size(g, s, weight='weight')

17

In [145]:
x.transpose() @ Q @ x

-672.0